In [113]:
import pandas as pd
from src.services.cases import get_cases, get_single_case
from src.models.cases import Case
from collections.abc import MutableMapping
import datetime

## Data Loading

In [103]:
cases = get_cases(
    start_date=datetime.datetime(2023, 9, 20), 
    end_date=datetime.datetime(2023, 9, 30)
)

/Users/aennassiri/Projects/Personal/ticket-washer/venv/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/Users/aennassiri/Projects/Personal/ticket-washer/src/services/cases.py:35: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  cases_list = cases_list.where(


In [104]:
cases_deep_copy = [case.model_copy() for case in cases]

# cases = [case.model_copy() for case in cases_deep_copy]

In [105]:

def flatten(dictionary, parent_key="", separator="_"):
    items = []
    for key, value in dictionary.items():
        new_key = parent_key + separator + key if parent_key else key
        if isinstance(value, MutableMapping):
            items.extend(flatten(value, new_key, separator=separator).items())
        else:
            items.append((new_key, value))
    return dict(items)


In [106]:
def get_case_dict(case: Case):
    court_date = None
    court_time = None
    case_data = flatten(case.model_dump())

    # Transform case location
    location = case.location
    if location is None:
        location = ""
    if "municipal" in location.lower():
        case_data["city"] = location.lower().replace("municipal", "").upper()
        case_data["city"] += "CITY"
        case_data["location"] = "MUNICIPAL"
    elif "circuit" in location.lower():
        case_data["city"] = location.lower().replace("circuit", "").upper()
        case_data["city"] += "COUNTY"
        case_data["location"] = "CIRCUIT"

    # Adding the current date short
    case_data["current_date_short"] = (
        datetime.datetime.now().strftime("%B %d, %Y").upper()
    )

    # Adding charges 
    charges = case_data.get("charges", [{"charge_description": ""}])
    if charges:
        case_data["charges_description"] = charges[0].get("charge_description", "")
    else: 
        case_data["charges_description"] = ""

    # Adding the court date and time
    if case.dockets is not None:
        for docket in case.dockets:
            if "initial" in docket.get("docket_desc", "").lower():
                # Get the associated_docketscheduledinfo
                schedule = docket.get("associated_docketscheduledinfo", {})
                if isinstance(schedule, list) and len(schedule) > 0:
                    schedule = schedule.pop()
                else:
                    schedule = {}
                court_date = schedule.get("associated_date", "")
                court_time = schedule.get("associated_time", "")
                break

    judge = None

    if case.judge is not None:
        judge = case.judge.get("formatted_name")
    output = {
        "case_id": case.case_id,
        "first_name": case.first_name,
        "last_name": case.last_name,
        "middle_name": case.middle_name,
        "location": case.location,
        "birth_date": case.birth_date,
        "court_date": court_date,
        "court_time": court_time,
        "judge": judge,
        "current_date": case_data.get("current_date_short", ""),
        "city": case_data.get("city", ""),
        "location": case_data.get("location", ""),
        "location_original": location,
        "charges_description": case_data.get("charges_description", ""),
    }
    return output

In [107]:
cases_df = pd.DataFrame([get_case_dict(c) for c in cases])

In [108]:
cases_df.city.value_counts()

city
                         2197
INDEPENDENCE CITY         476
SPRINGFIELD CITY          303
LEE'S SUMMIT CITY         268
MARYLAND HEIGHTS CITY     195
                         ... 
NORTHMOOR CITY              1
ANDERSON CITY               1
NIXA CITY                   1
TRUESDALE CITY              1
BRANSON  DIVISIONCITY       1
Name: count, Length: 195, dtype: int64

In [109]:
cases_df

,case_id,first_name,last_name,middle_name,location,birth_date,court_date,court_time,judge,current_date,city,location_original,charges_description
0,120266318,JEFFREY,YAVROUIAN,MELVIN,MUNICIPAL,10/16/1986,10/24/2023,1:00 PM,"ROSE, RICHARD EDWARDS","SEPTEMBER 28, 2023",GLADSTONE CITY,Gladstone Municipal,Miscellaneous Animal Violation
1,130118390,AMANDA,JOHNSTON,MARIE,MUNICIPAL,06/12/1984,10/03/2023,10:00 AM,"CARTER, MICHAEL E","SEPTEMBER 28, 2023",WENTZVILLE CITY,Wentzville Municipal,Building Code Violation
2,140942346,JATUAN,HAMPTON,MARTRIECE,MUNICIPAL,11/17/1985,10/02/2023,2:00 PM,"BILLINGS, CHARLES H.","SEPTEMBER 28, 2023",DES PERES CITY,Des Peres Municipal,Display Unlawful Plates
3,140942384,NICHOLAS,HAILY,ANDRE,MUNICIPAL,05/11/1997,11/06/2023,2:00 PM,"BILLINGS, CHARLES H.","SEPTEMBER 28, 2023",DES PERES CITY,Des Peres Municipal,Stealing/Larceny/Theft
4,140942437,RYLAN,ROWEN,JAMES,MUNICIPAL,07/20/2005,11/06/2023,2:00 PM,"BILLINGS, CHARLES H.","SEPTEMBER 28, 2023",DES PERES CITY,Des Peres Municipal,Fail To Stop At Stop Sign At Stop Line/Before ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7636,CHURCHWELL__LELA_A_44_09_28_2023,LELA,CHURCHWELL,A,None,None,None,None,None,"SEPTEMBER 28, 2023",,,
7637,CONEJO__VIRIDIANA_S_22_09_28_2023,VIRIDIANA,CONEJO,S,None,None,None,None,None,"SEPTEMBER 28, 2023",,,
7638,GALLAGHER__SHANNAN_M_26_09_28_2023,SHANNAN,GALLAGHER,M,None,None,None,None,None,"SEPTEMBER 28, 2023",,,
7639,MCDANIELS__CARNITA_E_31_09_28_2023,CARNITA,MCDANIELS,E,None,None,None,None,None,"SEPTEMBER 28, 2023",,,


In [110]:
cases_df.describe()

,case_id,first_name,last_name,middle_name,location,birth_date,court_date,court_time,judge,current_date,city,location_original,charges_description
count,7641,7604,7641,6687,7452,7365,7378,7378,7402,7641,7641,7641,7641
unique,7641,2439,4215,1613,93,5154,75,30,229,1,195,286,230
top,120266318,MICHAEL,SMITH,MARIE,MUNICIPAL,01/01/0001,11/01/2023,9:00 AM,"HELM, GARRY LEE","SEPTEMBER 28, 2023",,Independence Municipal,Expired Plates
freq,1,119,77,190,5167,136,456,2520,346,7641,2197,476,557


In [111]:
cases_df.to_csv("cases.csv", index=False)

In [114]:
case  = get_single_case("161173646")

In [116]:
case.model_dump_json()

'{"case_id":"161173646","court_id":"SMPDB0001_CT07","participants":null,"related_cases":null,"protection_order":false,"parties":[{"other_attorneylist":[],"pred_code":"ZZ","desc":"Defendant","pty_seqno":2,"suffix":"JR","birth_date":"09/09/2003","lit_ind":"Y","mid_initial":"RAYE","pidm":90458616,"addr_line1":"120 SHERRI LANE","addr_city":"EXCELSIOR SPRINGS","birth_datecode":"Y","desc_code":"DFT","formatted_partyaddress":"120 SHERRI LANE\\nEXCELSIOR SPRINGS, MO 64024","last_name":"SHAW","addr_statcode":"MO","co_attorneylist":[],"formatted_telephone":"","attorney_list":[],"addr_atyp":"MA","formatted_birthdate":"2003","prosecuting_atty":false,"criminal_ind":"N","sort_seq":4,"addr_seqno":2,"assoc":0,"addr_zip":"64024","first_name":"ALTON","party_type":"party","formatted_partyname":"SHAW, ALTON RAYE JR","criminal_case":false},{"other_attorneylist":[],"pred_code":"ZZ","desc":"Assistant Prosecuting Attorney","pty_seqno":1,"tel_code":"BU","tel_area":"816","mid_initial":"CHARLES","pidm":90130406,